In [11]:
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pylab as pl
from preprocess import *

In [12]:
def box_count(image, box_size):
  """
  Counts the number of non-empty boxes of a specific size covering the image.

  Args:
    image: A 2D numpy array representing the binary image.
    box_size: The size of the square boxes used for counting.

  Returns:
    The number of non-empty boxes.
  """
  n_rows, n_cols = image.shape
  n_boxes_x = int(n_cols // box_size)
  n_boxes_y = int(n_rows // box_size)
  count = 0
  resized_image = image[:n_boxes_y * box_size, :n_boxes_x * box_size]
    
    # Reshape the resized image into boxes
  boxes = resized_image.reshape(n_boxes_y, box_size, n_boxes_x, box_size)
    
    # Check if any pixel in each box is non-zero (foreground)
  non_zero_boxes = np.any(boxes, axis=(1, 3))
    
    # Count the number of non-zero boxes
  count = np.sum(non_zero_boxes)
  return count
def box_counting_dimension(image, min_size, max_size, scale_factor=2):
  """
  Calculates the box counting dimension of a binary image.

  Args:
    image: A 2D numpy array representing the binary image.
    min_size: The minimum size of the square boxes used for counting.
    max_size: The maximum size of the square boxes used for counting.
    scale_factor: The factor by which the box size is scaled at each iteration.

  Returns:
    The estimated box counting dimension.
  """
  # Convert image to binary (foreground = 1, background = 0)
  image = image > 0  

  # Prepare lists to store box sizes and counts
  box_sizes = []
  box_counts = []
  
  # Iterate through different box sizes
  box_size = min_size
  while box_size <= max_size:
    count = box_count(image, box_size)
    box_sizes.append(box_size)
    box_counts.append(count)
    box_size *= scale_factor

  # Fit a linear regression to log(box_size) vs log(box_count)
  log_sizes = np.log(box_sizes)
  log_counts = np.log(box_counts)
  slope, _ = np.polyfit(log_sizes, log_counts, 1)
  print (slope)
  # Estimated box counting dimension is negative of the slope
  return -slope

In [13]:
A = load_Dataset("../fonts-dataset/Scheherazade New/*.jpeg")
B= load_Dataset("../fonts-dataset/Lemonada/*.jpeg")
C= load_Dataset("../fonts-dataset/Marhey/*.jpeg")
D= load_Dataset("../fonts-dataset/IBM Plex Sans Arabic/*.jpeg")

KeyboardInterrupt: 

In [ ]:
A_PROCESSED = []
for img in A:
    img = threshold_image(img)
    img= assure_white_bg(img)
    A_PROCESSED.append(img)
B_PROCESSED = []
for img in B:
    img = threshold_image(img)
    img= assure_white_bg(img)
    B_PROCESSED.append(img)
C_PROCESSED = []
for img in C:
    img = threshold_image(img)
    img= assure_white_bg(img)
    C_PROCESSED.append(img)
D_PROCESSED = []
for img in D:
    img = threshold_image(img)
    img= assure_white_bg(img)
    D_PROCESSED.append(img)

In [16]:
A_BCD=[]
for img in A_PROCESSED:
    A_BCD.append(box_counting_dimension(img,2,120))
print("A done")
B_BCD=[]
for img in B_PROCESSED:
    B_BCD.append(box_counting_dimension(img,2,120))
print("B done")
C_BCD=[]
for img in C_PROCESSED:
    C_BCD.append(box_counting_dimension(img,2,120))
print("C done")
D_BCD=[]
for img in D_PROCESSED:
    D_BCD.append(box_counting_dimension(img,2,120))
print("D done")
A_BCD=[result for result in A_BCD if not np.isnan(result)]
B_BCD=[result for result in B_BCD if not np.isnan(result)]
C_BCD=[result for result in C_BCD if not np.isnan(result)]
D_BCD=[result for result in D_BCD if not np.isnan(result)]

-1.3416258034420236
-1.28020393347032
-1.5086043607169133
-1.340927123547677
-1.3930107403966139
-1.273322475316328
-1.3394820320356724
-1.3637500073241093
-1.3380069797725465
-1.5133960011134617
-1.2735114933323426
-1.2705852596309601
-1.432598620329043
-1.3901559685823008
-1.2505621908120765
-1.321682890865267
-1.3483270012535407
-1.2671934532135283
-1.3129238006944657
-1.3440129193065287
-1.514894335934421
-1.3385509123659847
-1.3277251558956313
-1.2491155651643246
-1.343036324623426
-1.452238298646836
-1.3039773785961608
-1.3688734522542891
-1.4112166817703518
-1.3571535553868859
-1.299158488942883
-1.3308202834288285
-1.3217972338177226
-1.3486686016390748
-1.2881672043818735
-1.2670115809317652
-1.2536049728809389
-1.3535553185345064
-1.4332541839876975
-1.260468389862351
-1.3136568307840832
-1.3560615109352876
-1.407575797175966
-1.3127663035588568
-1.3286716220054375
-1.2989488861212064
-1.3162870466044454
-1.3696255590643043
-1.3623177830902404
-1.2486455454235317
-1.421929107

C:\Users\omara\AppData\Local\Temp\ipykernel_14044\3645010811.py:57: RuntimeWarning: divide by zero encountered in log
  log_counts = np.log(box_counts)


-1.4875103779616268
-1.6144505693313633
-1.6246268614314021
-1.4817365390850454
-1.4815474592259619
-1.4685035430349427
-1.3772090304712996
-1.4940510678361663
-1.4393829076390696
-1.4190095888272387
-1.494059708730875
-1.5746673830112854
-1.4569027156845742
-1.497660610683969
-1.3961510986628445
-1.458902953758611
-1.540325814673582
-1.421390074983575
-1.4617710401816302
-1.3997798450823804
-1.3961396027502457
-1.5549017511680399
-1.577371310208141
-1.4267706787705645
-1.3794081888168785
-1.3971289928740362
-1.4583440089989286
-1.6016252644591105
-1.3898825681756286
-1.480125378189919
-1.3219692541749193
-1.4512498968891474
-1.4671582183021497
-1.4486103195715319
-1.4525687352599304
-1.3928099605686188
-1.6638617403555438
-1.4828869036973507
-1.5848453025591294
-1.399229861733448
-1.4818517443396615
-1.5142617931026698
-1.566573000443973
-1.425604094607564
-1.4531539803771591
-1.4862752659160754
-1.5646515987590195
-1.467342659535075
-1.4164448655602142
-1.455281534601332
-1.485201764

In [17]:
X = np.concatenate([A_BCD, B_BCD, C_BCD, D_BCD]).reshape(-1,1)
y = np.concatenate([np.zeros(len(A_BCD)), np.ones(len(B_BCD)), 
                    2*np.ones(len(C_BCD)), 3*np.ones(len(D_BCD))]).reshape(-1,1)
train_features, test_features, train_labels, test_labels = train_test_split(
        X, y, test_size=0.3, random_state=40)
    
    #print(labels)
KNN = KNeighborsClassifier(n_neighbors=5)

KNN.fit(train_features, train_labels)
        
    
accuracy = KNN.score(test_features, test_labels)
        
print('accuracy: ', accuracy*100, '%')

accuracy:  42.70833333333333 %


C:\Users\omara\AppData\Roaming\Python\Python311\site-packages\sklearn\neighbors\_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
